## Resnet50 TensorFlow Pre-Trained Model Deployment With SageMaker Migration Toolkit

- [SageMaker Migration Toolkit]()
- [Resnet50 Model]()
- Notebook Setting: conda_tf2_p38 Kernel
- If you would like to deploy the model without using the API to abstract boto3 calls check the following [example](https://github.com/RamVegiraju/SageMaker-Deployment/tree/master/Misc/resnet-tf-sme).

## API Setup

1. Clone GitHub repo
2. python setup.py bdist_wheel
3. pip install dist/sagemaker_migration_toolkit-0.0.1-py3-none-any.whl
4. sagemaker_migration-configure --module-name sagemaker_migration.configure

Step 4 is where you provide your role for sagemaker execution, do this via the terminal in your notebook path. To get the role you are currently using you can also run ```sagemaker.get_execution_role()```.

Alternatively you can also install the package after cloning the repo with the following command in the notebook.

In [1]:
!pip install sagemaker-migration-toolkit/dist/sagemaker_migration_toolkit-0.0.1-py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Processing ./sagemaker-migration-toolkit/dist/sagemaker_migration_toolkit-0.0.1-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 80.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.24.19
    Uninstalling botocore-1.24.19:
      Successfully uninstalled botocore-1.24.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.5 requires botocore==1.29.5, but you have botocore 1.29.15 which is incompatible.
aiobotocore 2.0.1 requires botocore<1.22.9,>=1.22.8, but you have botocore 1.29.15 which is incompatible.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p38/bin/python -m pip install --upgrade pip' command.


## Resnet50 Model Artifacts

The API expects the model data to be serialized in the format compliant with the framework's serving stack. For [TensorFlow Serving](https://towardsdatascience.com/hosting-models-with-tf-serving-on-docker-aceff9fbf533) it should be the following structure.

```
model_artifact_version/
    assets
    variables/
        variables.data
        variables.index
    keras_metadata.pb
    saved_model.pb 
inference.py
```

For our example we don't need an inference.py, but this is mandatory for the package so we will make a dummy script. In the next few cells we will create the model artifacts for the pre-trained resnet50 model in the structure TensorFlow Serving expects.

In [2]:
import os
import tensorflow as tf
from tensorflow.keras.applications import resnet50
from tensorflow.keras import backend
import numpy as np
from tensorflow.keras.preprocessing import image
import sagemaker
role = sagemaker.get_execution_role()
role #pass this role in step 4 of API setup

'arn:aws:iam::474422712127:role/sagemaker-role-BYOC'

In [3]:
tf.keras.backend.set_learning_phase(0)
model = resnet50.ResNet50()

/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/keras/backend.py:414: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '
2022-11-22 20:41:03.270035: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-22 20:41:03.270098: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-16-96-163.ec2.internal): /proc/driver/nvidia/version does not exist
2022-11-22 20:41:03.270842: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-crit

102981632/102967424 [==============================] - 0s 0us/step


In [4]:
export_dir = "00002" #directory to store model artifacts
model = tf.keras.applications.ResNet50()

if not os.path.exists(export_dir):
    os.makedirs(export_dir)
    print("Directory ", export_dir, " Created ")
else:
    print("Directory ", export_dir, " already exists")
# Save to SavedModel
model.save(export_dir, save_format="tf", include_optimizer=False)

Directory  00002  Created 


2022-11-22 20:41:20.617096: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: 00002/assets


/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


Here we will create a dummy inference.py script, you should replace this with your actual pre and post processing as necessary

In [5]:
%%writefile inference.py
import os
import json

Writing inference.py


## SageMaker Migration Toolkit Deployment

In [6]:
from sagemaker_migration import frameworks as fwk
import os

In [7]:
# Create a TensorFlow Model Object Entity, you can create a real-time or serverless endpoint
tf_model = fwk.TensorFlowModel(
    version = '2.3.0',
    model_data = '00002',
    inference_option = 'real-time',
    inference = 'inference.py',
    instance_type = 'ml.m5.xlarge')

In [8]:
tf_model.deploy_to_sagemaker()

Model data contains:  00002
['assets', 'keras_metadata.pb', 'variables', 'saved_model.pb']
00002
inference.py
model.tar.gz
Uploading model.tar.gz to S3...
Creating model in SageMaker...
Created model: sm-model-tensorflow-2022-11-22-20-42-30
Creating endpoint config in SageMaker...
Created endpoint config: sm-endpoint-config-tensorflow-2022-11-22-20-42-30
Creating endpoint in SageMaker...
Endpoint creation in process: sm-endpoint-tensorflow-2022-11-22-20-42-30
Monitoring endpoint creation...
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService


{'EndpointName': 'sm-endpoint-tensorflow-2022-11-22-20-42-30',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:474422712127:endpoint/sm-endpoint-tensorflow-2022-11-22-20-42-30',
 'EndpointConfigName': 'sm-endpoint-config-tensorflow-2022-11-22-20-42-30',
 'ProductionVariants': [{'VariantName': 'primaryvariant',
   'DeployedImages': [{'SpecifiedImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.3.0-cpu',
     'ResolvedImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference@sha256:91ebb7428846c5f7b515d5d9b8389a14c73d0c5d02657f4a6413592124333278',
     'ResolutionTime': datetime.datetime(2022, 11, 22, 20, 42, 31, 650000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2022, 11, 22, 20, 42, 31, 147000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 11, 22, 20, 44, 9, 1000, tzinfo=tzl

## Inference

In [13]:
import boto3
#runtime client for inference
runtime = boto3.client(service_name="sagemaker-runtime") #runtime boto3 client for inference
#grab endpoint name
endpoint_name = 'sm-endpoint-tensorflow-2022-11-22-20-42-30'

You can retrieve the data from the public Amazon datasets with the following curl command. I've also posted the same image in the repo incase you cannot retrieve.

```
file_name = 'kitten.jpg'
!wget -q https://s3.amazonaws.com/model-server/inputs/kitten.jpg -O {file_name}
```

In [11]:
import numpy as np
from tensorflow.keras.preprocessing import image

# preprocess image
img = image.load_img("kitten.jpg", target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = resnet50.preprocess_input(x)
print(type(x))

<class 'numpy.ndarray'>


In [12]:
import json
payload = json.dumps(x.tolist())
#payload

In [15]:
response = runtime.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType='application/json',
                                   Body=payload)
result = json.loads(response['Body'].read().decode())
#result